In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except Exception:
    HAS_XGB = False

import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from sqlalchemy import create_engine
DB_USER = "postgres"
DB_PASS = "142789"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "Medical_Insurance_Cost_Prediction_db"
TABLE_NAME = "Medical_Insurance"

engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
df = pd.read_sql(f'SELECT * FROM "Medical_Insurance";', engine)
for col in ["age", "bmi", "children", "charges"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df = df.dropna(subset=["charges"]).reset_index(drop=True)
# 1. Features / Target
# -----------------------------
target = "charges"
cat_cols = ["sex", "smoker", "region"]
num_cols = [c for c in df.columns if c not in cat_cols + [target]]

X = df[num_cols + cat_cols].copy()
y = df[target].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 2. Preprocessor
# -----------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="passthrough"
)


# 3. Candidate Models
# -----------------------------
pipelines = {
    "LinearRegression": Pipeline([
        ("prep", preprocessor),
        ("model", LinearRegression())
    ]),
    "RandomForest": Pipeline([
        ("prep", preprocessor),
        ("model", RandomForestRegressor(n_estimators=300, random_state=42))
    ]),
}

if HAS_XGB:
    pipelines["XGBoost"] = Pipeline([
        ("prep", preprocessor),
        ("model", XGBRegressor(
            n_estimators=400, max_depth=5, learning_rate=0.05,
            subsample=0.9, colsample_bytree=0.9, random_state=42
        ))
    ])

# 4. MLflow experiment
# -----------------------------
mlflow.set_experiment("Medical Insurance Models")

best_name, best_r2, best_pipe = None, -1e9, None

for name, pipe in pipelines.items():
    with mlflow.start_run(run_name=name):
        pipe.fit(X_train, y_train)
        preds = pipe.predict(X_test)

        mae = mean_absolute_error(y_test, preds)
        mse = mean_squared_error(y_test, preds)
        r2  = r2_score(y_test, preds)

        mlflow.log_param("model_name", name)
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("R2",  r2)

        input_example = X_train.iloc[:1]
        signature = infer_signature(input_example, pipe.predict(input_example))

        mlflow.sklearn.log_model(
            sk_model=pipe,
            artifact_path="model",
            input_example=input_example,
            signature=signature
        )

        # Track best
        if r2 > best_r2:
            best_r2 = r2
            best_name = name
            best_pipe = pipe

print(f"Best model: {best_name} (R²={best_r2:.4f})")

payload = {
    "pipeline": best_pipe,
    "feature_order": X.columns.tolist(),
}
joblib.dump(payload, "best_model.pkl")
print("✅ Saved best pipeline to best_model.pkl")

pip install psycopg2-binary sqlalchemy streamlit joblib pandas scikit-learn

%%writefile Medical_Insurance_Prediction.py

import streamlit as st
import pandas as pd
import joblib
import psycopg2
from sqlalchemy import create_engine

# -----------------------------
# Page setup
# -----------------------------
st.set_page_config(
    page_title="Medical Insurance Cost Prediction",
    page_icon="💊",
    layout="centered"
)

st.title("💊 Medical Insurance Cost Prediction")
st.write("Fill in the details below and get an estimated insurance charge.")

# -----------------------------
# Database setup
# -----------------------------
DB_USER = "postgres"
DB_PASS = "142789"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "Medical_Insurance_Cost_Prediction_db"
TABLE_NAME = "Medical_Insurance"

# Create DB engine
@st.cache_resource
def get_engine():
    return create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

engine = get_engine()

# -----------------------------
# Load trained pipeline
# -----------------------------
@st.cache_resource
def load_pipeline():
    """Load the trained pipeline from disk"""
    obj = joblib.load("best_model.pkl")
    if isinstance(obj, dict):
        return obj["pipeline"], obj.get("feature_order")
    return obj, None

try:
    pipeline, feature_order = load_pipeline()
except Exception as e:
    st.error(f"❌ Could not load `best_model.pkl` — {e}")
    st.stop()

# -----------------------------
# Input form
# -----------------------------
with st.form("insurance_form", clear_on_submit=False):
    col1, col2 = st.columns(2)

    with col1:
        age = st.number_input("👤 Age", min_value=18, max_value=100, value=30, step=1)
        bmi = st.number_input("⚖️ BMI", min_value=10.0, max_value=60.0, value=25.0, step=0.1)
        children = st.number_input("👶 Number of Children", min_value=0, max_value=10, value=0, step=1)

    with col2:
        sex = st.radio("⚧ Sex", ["male", "female"], horizontal=True)
        smoker = st.radio("🚬 Smoker", ["no", "yes"], horizontal=True)
        region = st.selectbox("🌍 Region", ["northeast", "northwest", "southeast", "southwest"])

    submitted = st.form_submit_button("🔮 Predict Insurance Cost")

# -----------------------------
# Prediction + Logging
# -----------------------------
if submitted:
    # Prepare input data
    input_data = pd.DataFrame([{
        "age": age,
        "bmi": bmi,
        "children": children,
        "sex": sex,
        "smoker": smoker,
        "region": region,
    }])

    if feature_order is not None:
        input_data = input_data.reindex(columns=feature_order)

    try:
        # Predict cost
        prediction = pipeline.predict(input_data)[0]

        # Show result
        st.subheader("✅ Prediction Result")
        st.success(f"💵 Estimated Insurance Cost: **${prediction:,.2f}**")

        # Save to PostgreSQL
        log_df = input_data.copy()
        log_df["predicted_cost"] = prediction
        log_df.to_sql(TABLE_NAME, engine, if_exists="append", index=False)

        st.info("🗄️ This prediction was logged into the database.")

    except Exception as e:
        st.error(f"⚠️ Prediction failed: {e}")
        st.write("Here is the input data we tried to predict with:")
        st.dataframe(input_data)

# -----------------------------
# Optional: Show past predictions
# -----------------------------
if st.checkbox("📊 Show previous predictions from database"):
    try:
        # User controls
        num_rows = st.slider("How many records to show?", min_value=5, max_value=50, value=10, step=5)
        sort_by = st.selectbox("Sort by:", ["age", "predicted_cost"], index=0)

        # Query database
        query = f'SELECT * FROM "Medical_Insurance" ORDER BY "{sort_by}" DESC LIMIT {num_rows};'
        history = pd.read_sql(query, engine)

        st.subheader("📋 Recent Predictions from Database")
        st.dataframe(history)

    except Exception as e:
        st.error(f"Could not fetch past predictions: {e}")
!streamlit run  Medical_Insurance_Prediction.py
%%writefile Medical_Insurance_Prediction_2.py

import streamlit as st
import pandas as pd
import joblib
import psycopg2
from sqlalchemy import create_engine
import io
import datetime

# -----------------------------
# Page setup
# -----------------------------
st.set_page_config(
    page_title="Medical Insurance Cost Prediction",
    page_icon="💊",
    layout="centered"
)

st.title("💊 Medical Insurance Cost Prediction")
st.write("Fill in the details below and get an estimated insurance charge.")

# -----------------------------
# Database setup
# -----------------------------
DB_USER = "postgres"
DB_PASS = "142789"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "Medical_Insurance_Cost_Prediction_db"
TABLE_NAME = "Medical_Insurance"

# Create DB engine
@st.cache_resource
def get_engine():
    return create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

engine = get_engine()

# -----------------------------
# Load trained pipeline
# -----------------------------
@st.cache_resource
def load_pipeline():
    """Load the trained pipeline from disk"""
    obj = joblib.load("best_model.pkl")
    if isinstance(obj, dict):
        return obj["pipeline"], obj.get("feature_order")
    return obj, None

try:
    pipeline, feature_order = load_pipeline()
except Exception as e:
    st.error(f"❌ Could not load `best_model.pkl` — {e}")
    st.stop()

# -----------------------------
# Input form
# -----------------------------
with st.form("insurance_form", clear_on_submit=False):
    col1, col2 = st.columns(2)

    with col1:
        age = st.number_input("👤 Age", min_value=18, max_value=100, value=30, step=1)
        bmi = st.number_input("⚖️ BMI", min_value=10.0, max_value=60.0, value=25.0, step=0.1)
        children = st.number_input("👶 Number of Children", min_value=0, max_value=10, value=0, step=1)

    with col2:
        sex = st.radio("⚧ Sex", ["male", "female"], horizontal=True)
        smoker = st.radio("🚬 Smoker", ["no", "yes"], horizontal=True)
        region = st.selectbox("🌍 Region", ["northeast", "northwest", "southeast", "southwest"])

    submitted = st.form_submit_button("🔮 Predict Insurance Cost")

# -----------------------------
# Prediction + Logging
# -----------------------------
if submitted:
    # Prepare input data
    input_data = pd.DataFrame([{
        "age": age,
        "bmi": bmi,
        "children": children,
        "sex": sex,
        "smoker": smoker,
        "region": region,
    }])

    if feature_order is not None:
        input_data = input_data.reindex(columns=feature_order)

    try:
        # Predict cost
        prediction = pipeline.predict(input_data)[0]

        # Show result
        st.subheader("✅ Prediction Result")
        st.success(f"💵 Estimated Insurance Cost: **${prediction:,.2f}**")

        # Save to PostgreSQL with timestamp
        log_df = input_data.copy()
        log_df["predicted_cost"] = prediction
        log_df["created_at"] = datetime.datetime.now()
        log_df.to_sql(TABLE_NAME, engine, if_exists="append", index=False)

        st.info("🗄️ This prediction was logged into the database.")

    except Exception as e:
        st.error(f"⚠️ Prediction failed: {e}")
        st.write("Here is the input data we tried to predict with:")
        st.dataframe(input_data)

# -----------------------------
# Optional: Show past predictions
# -----------------------------
if st.checkbox("📊 Show previous predictions from database"):
    try:
        # -----------------------------
        # User controls
        # -----------------------------
        st.markdown("### 🔎 Filter Results")

        col1, col2, col3 = st.columns(3)

        with col1:
            filter_sex = st.selectbox("⚧ Sex", ["All", "male", "female"])
        with col2:
            filter_smoker = st.selectbox("🚬 Smoker", ["All", "yes", "no"])
        with col3:
            filter_region = st.selectbox("🌍 Region", ["All", "northeast", "northwest", "southeast", "southwest"])

        num_rows = st.slider("How many records to show?", min_value=5, max_value=50, value=10, step=5)
        sort_by = st.selectbox("Sort by:", ["created_at", "age", "predicted_cost"], index=0)

        # -----------------------------
        # Build WHERE clause dynamically
        # -----------------------------
        conditions = []
        if filter_sex != "All":
            conditions.append(f"sex = '{filter_sex}'")
        if filter_smoker != "All":
            conditions.append(f"smoker = '{filter_smoker}'")
        if filter_region != "All":
            conditions.append(f"region = '{filter_region}'")

        where_clause = " AND ".join(conditions)
        if where_clause:
            where_clause = "WHERE " + where_clause

        # -----------------------------
        # Query database
        # -----------------------------
        query = f'SELECT * FROM "{TABLE_NAME}" {where_clause} ORDER BY "{sort_by}" DESC LIMIT {num_rows};'
        history = pd.read_sql(query, engine)

        st.subheader("📋 Recent Predictions from Database")
        st.dataframe(history)

        # -----------------------------
        # KPI Cards
        # -----------------------------
        if not history.empty:
            avg_cost = history["predicted_cost"].mean()
            max_cost = history["predicted_cost"].max()
            min_cost = history["predicted_cost"].min()

            st.markdown("### 📊 Key Statistics")
            kpi1, kpi2, kpi3 = st.columns(3)

            with kpi1:
                st.metric(label="📈 Average Cost", value=f"${avg_cost:,.2f}")

            with kpi2:
                st.metric(label="💰 Highest Cost", value=f"${max_cost:,.2f}")

            with kpi3:
                st.metric(label="🏷️ Lowest Cost", value=f"${min_cost:,.2f}")

        # -----------------------------
        # Download Buttons
        # -----------------------------
        st.markdown("### 📥 Download Prediction History")

        csv_buffer = io.StringIO()
        history.to_csv(csv_buffer, index=False)
        st.download_button(
            label="⬇️ Download as CSV",
            data=csv_buffer.getvalue(),
            file_name="prediction_history.csv",
            mime="text/csv"
        )

        excel_buffer = io.BytesIO()
        with pd.ExcelWriter(excel_buffer, engine="xlsxwriter") as writer:
            history.to_excel(writer, index=False, sheet_name="Predictions")
        st.download_button(
            label="⬇️ Download as Excel",
            data=excel_buffer.getvalue(),
            file_name="prediction_history.xlsx",
            mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        )

        # -----------------------------
        # Clear History Button
        # -----------------------------
        st.markdown("### 🗑️ Manage History")
        if st.button("⚠️ Clear All Prediction History"):
            with engine.begin() as conn:
                conn.execute(f'DELETE FROM "{TABLE_NAME}";')
            st.warning("🗑️ All prediction history has been cleared! Refresh to see changes.")

    except Exception as e:
        st.error(f"Could not fetch past predictions: {e}")

!streamlit run Medical_Insurance_Prediction_2.py
from sqlalchemy import create_engine, text
from sqlalchemy import types as satypes

# ...
TABLE_NAME = "medical_insurance"   # <<— use lowercase to match Postgres default

# inside the try: block where you log to DB
try:
    # Predict cost
    prediction = pipeline.predict(input_data)[0]

    # Show result
    st.subheader("✅ Prediction Result")
    st.success(f"💵 Estimated Insurance Cost: **${prediction:,.2f}**")

    # Build row to insert
    log_df = input_data.copy()
    log_df["predicted_cost"] = float(prediction)    # ensure numeric
    log_df["created_at"] = pd.Timestamp.utcnow()    # store UTC

    # (Optional but robust) Tell Postgres the types we expect
    dtype_map = {
        "age": satypes.Integer(),
        "bmi": satypes.Numeric(6, 2),
        "children": satypes.Integer(),
        "sex": satypes.Text(),
        "smoker": satypes.Text(),
        "region": satypes.Text(),
        "predicted_cost": satypes.Numeric(12, 2),
        "created_at": satypes.DateTime(),
    }

    # Write and verify in a single transaction
    with engine.begin() as conn:
        log_df.to_sql(
            TABLE_NAME,
            conn,
            if_exists="append",
            index=False,
            dtype=dtype_map,
            method="multi",
            schema="public",  # explicit schema helps
        )
        # sanity check
        total = conn.execute(text(f'SELECT COUNT(*) FROM public.{TABLE_NAME}')).scalar()

    st.info(f"🗄️ Prediction logged. Table now has {total} rows.")

except Exception as e:
    # Show the real DB error so you can diagnose quickly
    st.error("⚠️ Prediction logged failed.")
    st.exception(e)
    st.write("Payload we tried to insert:")
    st.dataframe(log_df if "log_df" in locals() else input_data)

%%writefile Medical_Insurance_Prediction_3.py

import streamlit as st
import pandas as pd
import joblib
import psycopg2
from sqlalchemy import create_engine
import io
import datetime

# -----------------------------
# Page setup
# -----------------------------
st.set_page_config(
    page_title="Medical Insurance Cost Prediction",
    page_icon="💊",
    layout="centered"
)

st.title("💊 Medical Insurance Cost Prediction")
st.write("Fill in the details below and get an estimated insurance charge.")

# -----------------------------
# Database setup
# -----------------------------
DB_USER = "postgres"
DB_PASS = "142789"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "Medical_Insurance_Cost_Prediction_db"
TABLE_NAME = "Medical_Insurance"

# Create DB engine
@st.cache_resource
def get_engine():
    return create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

engine = get_engine()

# -----------------------------
# Load trained pipeline
# -----------------------------
@st.cache_resource
def load_pipeline():
    """Load the trained pipeline from disk"""
    obj = joblib.load("best_model.pkl")
    if isinstance(obj, dict):
        return obj["pipeline"], obj.get("feature_order")
    return obj, None

try:
    pipeline, feature_order = load_pipeline()
except Exception as e:
    st.error(f"❌ Could not load `best_model.pkl` — {e}")
    st.stop()

# -----------------------------
# Input form
# -----------------------------
with st.form("insurance_form", clear_on_submit=False):
    col1, col2 = st.columns(2)

    with col1:
        age = st.number_input("👤 Age", min_value=18, max_value=100, value=30, step=1)
        bmi = st.number_input("⚖️ BMI", min_value=10.0, max_value=60.0, value=25.0, step=0.1)
        children = st.number_input("👶 Number of Children", min_value=0, max_value=10, value=0, step=1)

    with col2:
        sex = st.radio("⚧ Sex", ["male", "female"], horizontal=True)
        smoker = st.radio("🚬 Smoker", ["no", "yes"], horizontal=True)
        region = st.selectbox("🌍 Region", ["northeast", "northwest", "southeast", "southwest"])

    submitted = st.form_submit_button("🔮 Predict Insurance Cost")

# -----------------------------
# Prediction + Logging
# -----------------------------
from sqlalchemy import create_engine, text
from sqlalchemy import types as satypes

# ...
TABLE_NAME = "Medical_Insurance"   # <<— use lowercase to match Postgres default

# inside the try: block where you log to DB
if submitted:
    # -----------------------------
    # Build the input DataFrame
    # -----------------------------
    input_data = pd.DataFrame([{
        "age": age,
        "bmi": bmi,
        "children": children,
        "sex": sex,
        "smoker": smoker,
        "region": region,
    }])

    if feature_order is not None:
        input_data = input_data.reindex(columns=feature_order)

    # -----------------------------
    # Predict and log to DB
    # -----------------------------
    from sqlalchemy import text, types as satypes
    try:
        # Predict cost
        prediction = pipeline.predict(input_data)[0]

        # Show result
        st.subheader("✅ Prediction Result")
        st.success(f"💵 Estimated Insurance Cost: **${prediction:,.2f}**")

        # Build row to insert
        log_df = input_data.copy()
        log_df["predicted_cost"] = float(prediction)    # ensure numeric
        log_df["created_at"] = pd.Timestamp.utcnow()    # UTC timestamp

        # Explicit column type mapping
        dtype_map = {
            "age": satypes.Integer(),
            "bmi": satypes.Numeric(6, 2),
            "children": satypes.Integer(),
            "sex": satypes.Text(),
            "smoker": satypes.Text(),
            "region": satypes.Text(),
            "predicted_cost": satypes.Numeric(12, 2),
            "created_at": satypes.DateTime(),
        }

        # Write + verify
        with engine.begin() as conn:
            log_df.to_sql(
                TABLE_NAME,
                conn,
                if_exists="append",
                index=False,
                dtype=dtype_map,
                method="multi",
                schema="public",
            )
            total = conn.execute(text(f'SELECT COUNT(*) FROM public.{TABLE_NAME}')).scalar()

        st.info(f"🗄️ Prediction logged. Table now has {total} rows.")

    except Exception as e:
        st.error("⚠️ Prediction logged failed.")
        st.exception(e)
        st.write("Payload we tried to insert:")
        st.dataframe(log_df if "log_df" in locals() else input_data)
# -----------------------------
# Optional: Show past predictions
# -----------------------------
if st.checkbox("📊 Show previous predictions from database"):
    try:
        # -----------------------------
        # User controls
        # -----------------------------
        st.markdown("### 🔎 Filter Results")

        col1, col2, col3 = st.columns(3)

        with col1:
            filter_sex = st.selectbox("⚧ Sex", ["All", "male", "female"])
        with col2:
            filter_smoker = st.selectbox("🚬 Smoker", ["All", "yes", "no"])
        with col3:
            filter_region = st.selectbox("🌍 Region", ["All", "northeast", "northwest", "southeast", "southwest"])

        num_rows = st.slider("How many records to show?", min_value=5, max_value=50, value=10, step=5)
        sort_by = st.selectbox("Sort by:", ["created_at", "age", "predicted_cost"], index=0)

        # -----------------------------
        # Build WHERE clause dynamically
        # -----------------------------
        conditions = []
        if filter_sex != "All":
            conditions.append(f"sex = '{filter_sex}'")
        if filter_smoker != "All":
            conditions.append(f"smoker = '{filter_smoker}'")
        if filter_region != "All":
            conditions.append(f"region = '{filter_region}'")

        where_clause = " AND ".join(conditions)
        if where_clause:
            where_clause = "WHERE " + where_clause

        # -----------------------------
        # Query database
        # -----------------------------
        query = f'SELECT * FROM "{TABLE_NAME}" {where_clause} ORDER BY "{sort_by}" DESC LIMIT {num_rows};'
        history = pd.read_sql(query, engine)

        st.subheader("📋 Recent Predictions from Database")
        st.dataframe(history)

        # -----------------------------
        # KPI Cards
        # -----------------------------
        if not history.empty:
            avg_cost = history["predicted_cost"].mean()
            max_cost = history["predicted_cost"].max()
            min_cost = history["predicted_cost"].min()

            st.markdown("### 📊 Key Statistics")
            kpi1, kpi2, kpi3 = st.columns(3)

            with kpi1:
                st.metric(label="📈 Average Cost", value=f"${avg_cost:,.2f}")

            with kpi2:
                st.metric(label="💰 Highest Cost", value=f"${max_cost:,.2f}")

            with kpi3:
                st.metric(label="🏷️ Lowest Cost", value=f"${min_cost:,.2f}")

        # -----------------------------
        # Download Buttons
        # -----------------------------
        st.markdown("### 📥 Download Prediction History")

        csv_buffer = io.StringIO()
        history.to_csv(csv_buffer, index=False)
        st.download_button(
            label="⬇️ Download as CSV",
            data=csv_buffer.getvalue(),
            file_name="prediction_history.csv",
            mime="text/csv"
        )

        excel_buffer = io.BytesIO()
        with pd.ExcelWriter(excel_buffer, engine="xlsxwriter") as writer:
            history.to_excel(writer, index=False, sheet_name="Predictions")
        st.download_button(
            label="⬇️ Download as Excel",
            data=excel_buffer.getvalue(),
            file_name="prediction_history.xlsx",
            mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        )

        # -----------------------------
        # Clear History Button
        # -----------------------------
        st.markdown("### 🗑️ Manage History")
        if st.button("⚠️ Clear All Prediction History"):
            with engine.begin() as conn:
                conn.execute(f'DELETE FROM "{TABLE_NAME}";')
            st.warning("🗑️ All prediction history has been cleared! Refresh to see changes.")

    except Exception as e:
        st.error(f"Could not fetch past predictions: {e}")

!streamlit run Medical_Insurance_Prediction_3.py
